<a href="https://colab.research.google.com/github/janwaelty/Master-Thesis/blob/main/image_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd


# Define source and destination paths
source_path = "/content/drive/My Drive/Colab Notebooks/image_preprocess.ipynb"
destination_path = "/content/drive/My Drive/image_preprocess.ipynb"

  # Check if the source file exists before moving
if os.path.exists(source_path):
  # Move the file
  shutil.move(source_path, destination_path)
  print(f"Notebook moved to: {destination_path}")

# Path to the Excel file containing the dataset
artist_data_path = "/content/drive/My Drive/artist_graph_data.xlsx"
artwork_numeric_data = pd.read_excel(artist_data_path)









In [ ]:
def load_image_data(zip_path, image_path):

  with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()

    file_list = [f for f in file_list if not f.startswith('__MACOSX')]
    zip_ref.extractall(extract_path, members=file_list)
  print(f"Dateien entpackt in: {extract_path}")

In [ ]:
# Function to check if an image file exists
def check_image_existence(image_paths, base_dir):
    missing_images = []  # To store paths of missing images
    nans = 0
    invalid_url = 0
    for image_path in image_paths:
        # Skip if the path is not a valid string (e.g., NaN or None)
        if isinstance(image_path, str) and image_path.strip():  # Check if it's a valid non-empty string
            # Construct the full image path based on the base directory
            full_image_path = os.path.join(base_dir, image_path.lstrip('/'))  # Remove leading slash, if any

            # Check if the file exists
            if not os.path.exists(full_image_path):
                missing_images.append(full_image_path)  # Add to missing list if not found
                invalid_url += 1
        else:
            # Handle case where image_path is not a valid string (e.g., NaN or empty string)
            nans += 1
    print(f"number of nans: {nans}")
    print(f"missing images: {invalid_url}")
    return missing_images


# Get the image paths from the dataset
image_paths = artwork_numeric_data['image_url'].values  # Replace 'image_url' with your actual column name

# Define the base directory where images are stored in Colab
base_dir = '/content/data'

# Check for missing images
missing_images = check_image_existence(image_paths, base_dir)


In [ ]:
if __name__ == "__main__":

  zip_path = "/content/drive/My Drive/pic.zip"
  extract_path = "/content/data"  # Zielordner für die entpackten Bilder